In [1]:
from nltk.parse import ChartParser
from nltk.grammar import CFG
import nltk
import csv
import re
import pandas as pd
import argparse
from sklearn.metrics import classification_report
import numpy as np
import torch
import math

In [2]:
# parser = argparse.ArgumentParser(description='input file name')
# parser.add_argument('--input',required=True, help="Please add the input file name path and name")
# args = parser.parse_args()

# Read Data from input file given by args.
# df=pd.read_csv(args.input, sep='\t',index_col='id')

In [3]:
def predict(text, bert_model, bert_tokenizer):
    # Tokenized input
    # text = "[CLS] I got restricted because Tom reported my reply [SEP]"
    text = "[CLS] " + text + " [SEP]"
    tokenized_text = bert_tokenizer.tokenize(text)
    # text = "[CLS] Stir the mixture until it is done [SEP]"
        #masked_index = 4
    sentence_prob = 1
    for masked_index in range(1,len(tokenized_text)-1):
        # Mask a token that we will try to predict back with `BertForMaskedLM`
        masked_word = tokenized_text[masked_index]
        #tokenized_text[masked_index] = '[MASK]'
        # assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']
        # print (tokenized_text)

        # Convert token to vocabulary indices
        indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)
        # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
        # segments_ids = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
        length = len(tokenized_text)
        segments_ids = [0 for _ in range(length)]
        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        # If you have a GPU, put everything on cuda
        # tokens_tensor = tokens_tensor.to('cuda')
        # segments_tensors = segments_tensors.to('cuda')

        # Load pre-trained model (weights)
        # bert_model = BertForMaskedLM.from_pretrained('bert-large-uncased')
        # bert_model.eval()

        # If you have a GPU, put everything on cuda
        # tokens_tensor = tokens_tensor.to('cuda')
        # segments_tensors = segments_tensors.to('cuda')
        # bert_model.to('cuda')

        # Predict all tokens
        with torch.no_grad():
            predictions = bert_model(tokens_tensor, segments_tensors)

        predictions = torch.nn.functional.softmax(predictions, -1)

        index = bert_tokenizer.convert_tokens_to_ids([masked_word])[0]

        curr_prob = predictions[0, masked_index][index]
        # predict_list = predictions[0, masked_index]
        sentence_prob *= curr_prob
        #tokenized_text[masked_index] = masked_word
    return math.pow(sentence_prob, 1/(len(tokenized_text)-3))
    #return sentence_prob
'''
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('./tmp/swag_output/')
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('./tmp/swag_output/')
model.eval()
'''


"\n# Load pre-trained model tokenizer (vocabulary)\ntokenizer = BertTokenizer.from_pretrained('./tmp/swag_output/')\n# Load pre-trained model (weights)\nmodel = BertForMaskedLM.from_pretrained('./tmp/swag_output/')\nmodel.eval()\n"

# Task A

In [4]:
df=pd.read_csv('../ALL data/Training  Data/subtaskA_data_all.csv', sep=',',index_col='id')
df_label=pd.read_csv('../ALL data/Training  Data/subtaskA_answers_all.csv', sep=',',header=None,index_col=0)
df_label.columns=['label-false']
df=df.join(df_label)


In [5]:
df

,sent0,sent1,label-false
id,,,
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0
1,He drinks apple.,He drinks milk.,0
2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1
3,A mosquito stings me,I sting a mosquito,1
4,A niece is a person.,A giraffe is a person.,1
...,...,...,...
9995,Mark ate a big bitter cherry pie,Mark ate a big sweet cherry pie,0
9996,Gloria wears a cat on her head,Gloria wears a hat on her head,0
9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...,1


In [6]:
df.shape

(10000, 3)

In [7]:
# select specific part of datframe
df1=df.head()

In [8]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
      

In [9]:
for index, row in df1.iterrows():  
# tokenize the rows of dataframe (pos column)
    sentence_1 = row['sent0']
    sentence_2 = row['sent1']

    prob_1 = predict(sentence_1, bert_model=model, bert_tokenizer=tokenizer)
    prob_2 = predict(sentence_2, bert_model=model, bert_tokenizer=tokenizer) 
    df1.loc[index,'prob_1']=prob_1
    df1.loc[index,'prob_2']=prob_2
    df1.loc[index,'predicted_false']=np.argmin([prob_1, prob_2]).astype('int64')

    print(np.min([prob_1, prob_2]))
    print('-------------')



0.39989597379060515
-------------
0.42548061000793774
-------------
0.5156204540425442
-------------
0.4335789488698799
-------------
0.5957062266924752
-------------


In [10]:
df1

,sent0,sent1,label-false,prob_1,prob_2,predicted_false
id,,,,,,
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0,0.399896,0.768483,0.0
1,He drinks apple.,He drinks milk.,0,0.610041,0.425481,1.0
2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1,0.603129,0.515620,1.0
3,A mosquito stings me,I sting a mosquito,1,0.458191,0.433579,1.0
4,A niece is a person.,A giraffe is a person.,1,0.715048,0.595706,1.0


In [11]:
# print accuracy report
print(f"Accuracy is equal to: {((df1['label-false']==df1['predicted_false']).sum()/df1.shape[0])*100} %")

Accuracy is equal to: 80.0 %


In [12]:
# from transformers import GPT2Tokenizer, GPT2Model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [13]:
# from transformers import GPT2LMHeadModel

In [14]:
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# model.eval()
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# def score(sentence):
#     tokenize_input = tokenizer.tokenize(sentence)
#     tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
#     loss=model(tensor_input, labels=tensor_input)
#     return -loss[0] * len(tokenize_input)

# a=['there is a book on the desk',
#                 'there is a plane on the desk',
#                         'there is a book in the desk']
# print([score(i) for i in a])

In [15]:
# from transformers import GPT2LMHeadModel, GPT2TokenizerFast
# model_id = 'gpt2'
# # model_id = 'gpt2-large'
# model = GPT2LMHeadModel.from_pretrained(model_id)
# tokenizer = GPT2TokenizerFast.from_pretrained(model_id)